This notebook explores distribitional simliarity in a dataset of 10,000 Wikipedia articles (4.4M words), building high-dimensional, sparse representations for words from the distinct contexts they appear in.  These representations allow for analysis of the most similar words to a given query, and are interpretable with respect to the specific contexts that are most important for determining that two words are similar.

In [1]:
from collections import defaultdict, Counter
import math
import operator
import gzip

In [2]:
window=2
vocabSize=10000

In [3]:
filename="../data/wiki.10K.txt"

wiki_data=open(filename, encoding="utf-8").read().lower().split(" ")


In [4]:
# We'll only create word representation for the most frequent K words

def create_vocab(data):
    word_representations={}
    vocab=Counter()
    for i, word in enumerate(data):
        vocab[word]+=1

    topK=[k for k,v in vocab.most_common(vocabSize)]
    for k in topK:
        word_representations[k]=defaultdict(float)
    return word_representations

In [5]:
# word representation for a word = its unigram distributional context (the unigrams that show
# up in a window before and after its occurence)

def count_unigram_context(data, word_representations):
    for i, word in enumerate(data):
        if word not in word_representations:
            continue
        start=i-window if i-window > 0 else 0
        end=i+window+1 if i+window+1 < len(data) else len(data)
        for j in range(start, end):
            if i != j:
                word_representations[word][data[j]]+=1

In [6]:
def count_directional_context(data, word_representations):
    for i, word in enumerate(data):
        if word not in word_representations:
            continue
        start=i-window if i-window > 0 else 0
        end=i+window+1 if i+window+1 < len(data) else len(data)
        left="L: %s" % ' '.join(data[start:i])
        right="R: %s" % ' '.join(data[i+1:end])
        
        word_representations[word][left]+=1
        word_representations[word][right]+=1

In [7]:
# normalize a word represenatation vector that its L2 norm is 1.
# we do this so that the cosine similarity reduces to a simple dot product

def normalize(word_representations):
    for word in word_representations:
        total=0
        for key in word_representations[word]:
            total+=word_representations[word][key]*word_representations[word][key]
            
        total=math.sqrt(total)
        for key in word_representations[word]:
            word_representations[word][key]/=total
        

In [8]:
def dictionary_dot_product(dict1, dict2):
    dot=0
    for key in dict1:
        if key in dict2:
            dot+=dict1[key]*dict2[key]
    return dot

In [9]:
def find_sim(word_representations, query):
    if query not in word_representations:
        print("'%s' is not in vocabulary" % query)
        return None
    
    scores={}
    for word in word_representations:
        cosine=dictionary_dot_product(word_representations[query], word_representations[word])
        scores[word]=cosine
    return scores

In [10]:
# Find the K words with highest cosine similarity to a query in a set of word_representations
def find_nearest_neighbors(word_representations, query, K):
    scores=find_sim(word_representations, query)
    if scores != None:
        sorted_x = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)
        for idx, (k, v) in enumerate(sorted_x[:K]):
            print("%s\t%s\t%.5f" % (idx,k,v))

Explore the difference between `count_unigram_context` and `count_directional_context` for determining what counts as "context".  `count_unigram_context` counts an individual unigram in the bag of words around a target as a "context" variable, while `count_directional_context` counts the sequence of words before and after the word as a single "context"--and specifies the direction it occurs (to the left or right of the word).

In [11]:
word_representations=create_vocab(wiki_data)
count_directional_context(wiki_data, word_representations)
normalize(word_representations)

In [12]:
find_nearest_neighbors(word_representations, "actor", 10)

0	actor	1.00000
1	politician	0.54099
2	actress	0.52242
3	cricketer	0.42361
4	artist	0.40005
5	writer	0.38234
6	cyclist	0.36833
7	musician	0.33385
8	diplomat	0.32010
9	poet	0.31124


In [13]:
# Let's find the contexts shared between two words that have the most contribution
# to the cosine similarity

def find_shared_contexts(word_representations, query1, query2, K):
    if query1 not in word_representations:
        print("'%s' is not in vocabulary" % query1)
        return None
    
    if query2 not in word_representations:
        print("'%s' is not in vocabulary" % query2)
        return None
    
    context_scores={}
    dict1=word_representations[query1]
    dict2=word_representations[query2]
    
    for key in dict1:
        if key in dict2:
            score=dict1[key]*dict2[key]
            context_scores[key]=score

    sorted_x = sorted(context_scores.items(), key=operator.itemgetter(1), reverse=True)
    for idx, (k, v) in enumerate(sorted_x[:K]):
        print("%s\t%s\t%.5f" % (idx,k,v))

In [14]:
find_shared_contexts(word_representations, "actor", "politician", 10)

0	R: . he	0.21961
1	L: an american	0.13391
2	R: ) .	0.11417
3	R: in the	0.01410
4	L: an indian	0.00761
5	L: a canadian	0.00677
6	L: an english	0.00564
7	R: of the	0.00564
8	L: a french	0.00423
9	R: , who	0.00338


Q1: Fill out a function `scale_tfidf` below.  This function takes as input a dict of word_representations and scales the value for each context in `word_representations[word]` by its tf-idf score.  Use the term frequency for tf and ${N \over |\{d \in D : t \in d\}|}$ for idf.  Here, tf measure the count of a *context* term for a particular *word*, and idf measures the number of distinct *words* a particular *context* is seen with.  This function should modify `word_representations` in place.

In [15]:
def scale_tfidf(word_representations):
    # count IDFs
    idfs=defaultdict(int)
    for term in word_representations:
        for context in word_representations[term]:
            idfs[context]+=1
            
    N=len(word_representations)
    for word in word_representations:
        for key in word_representations[word]:
            word_representations[word][key]*=math.log(N/idfs[key])
            
            

In [16]:
tf_idf_word_representations=create_vocab(wiki_data)
count_directional_context(wiki_data, tf_idf_word_representations)
scale_tfidf(tf_idf_word_representations)
normalize(tf_idf_word_representations)

Q2: Compare the results the results of tf-idf scaling with the non-scaled results above.  How does scaling change the quality of the nearest neighbors, or the sensibility of the significant contexts?  Provide examples to support your claims using `find_nearest_neighbors` and `find_shared_contexts` below.

In [17]:
query="two"
find_nearest_neighbors(word_representations, query, 10)
print()
find_nearest_neighbors(tf_idf_word_representations, query, 10)

0	two	1.00000
1	three	0.68766
2	four	0.61315
3	term	0.54894
4	song	0.54867
5	church	0.54464
6	main	0.54448
7	company	0.54311
8	band	0.54142
9	building	0.54053

0	two	1.00000
1	three	0.42725
2	four	0.33787
3	five	0.31393
4	several	0.23219
5	six	0.22343
6	few	0.22313
7	eight	0.22192
8	seven	0.20937
9	twenty	0.18513


In [18]:
query1="two"
query2="three"
find_shared_contexts(word_representations, query1, query2, 10)
print()
find_shared_contexts(tf_idf_word_representations, query1, query2, 10)

0	L: of the	0.15490
1	L: . the	0.09570
2	R: of the	0.04114
3	L: the first	0.03932
4	L: , and	0.03102
5	L: there are	0.03037
6	R: years later	0.02998
7	L: , the	0.02032
8	L: one of	0.01649
9	R: years ,	0.01081

0	R: years later	0.05778
1	L: there are	0.02744
2	R: days later	0.01653
3	L: one of	0.01490
4	R: years ,	0.01324
5	R: years .	0.01194
6	L: the first	0.01144
7	R: or more	0.01066
8	L: the next	0.01057
9	L: divided into	0.01024
